# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [ ]:
# Import your libraries:

%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import sys
# print(sys.path)
# Adding a path so that it finds the package for numpy
sys.path.append('/usr/local/lib/python3.7/site-packages')
import numpy as np
import pandas as pd
from scipy import stats

# Challenge 1 - Import and Describe the Dataset

In this lab, we will use a dataset containing information about customer preferences. We will look at how much each customer spends in a year on each subcategory in the grocery store and try to find similarities using clustering.

The dataset is located [here](https://drive.google.com/file/d/1z1gYSD32ktbHuKSzB5JVS_u4YsLibh5F/view?usp=sharing), please download it and place it in the data folder.

In [ ]:
# loading the data:

customers = pd.read_csv('../data/Wholesale_customer_ data.csv')

#### Explore the dataset with mathematical and visualization techniques. What do you find?

Checklist:

* What does each column mean?
* Any categorical data to convert?
* Any missing data to remove?
* Column collinearity - any high correlations?
* Descriptive statistics - any outliers to remove?
* Column-wise data distribution - is the distribution skewed?
* Etc.

Additional info: Over a century ago, an Italian economist named Vilfredo Pareto discovered that roughly 20% of the customers account for 80% of the typical retail sales. This is called the [Pareto principle](https://en.wikipedia.org/wiki/Pareto_principle). Check if this dataset displays this characteristic.

In [ ]:
# Your code here:

customers.head()


In [ ]:
customers.info()

In [ ]:
def spot_nan(df):
    
    dict_count = {}
    
    for col in df.columns:
        count_nan = df[col].isna().sum()
        dict_count[col] = count_nan
        
    return dict_count

spot_nan(customers)

In [ ]:
plt.subplots(figsize=(10,10))

corr = customers.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
sns.heatmap(corr, mask=mask, annot = True, fmt='.1g')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
plt.subplots(figsize=(20,5))
sns.boxplot(x="variable", y="value", data=pd.melt(customers))


In [ ]:
summary = customers.describe()
summary = summary.round()
summary.transpose()

In [ ]:
customers.drop(['Channel'], axis=1, inplace=True)
customers.drop(['Region'], axis=1, inplace=True)

In [ ]:
outliers = customers[stats.zscore(customers) > 3]

outliers.drop_duplicates()

In [ ]:
len(outliers.drop_duplicates())

In [ ]:
for col in customers.columns:
    print(col)
    pareto = customers[customers[col] > customers[col].quantile(.80)]
    print(pareto[col].sum() / customers[col].sum())

In [ ]:
# Your observations here


# The database includes the sales by customer of a series of categories, plus two catgorical: Channel And Region
# No missing values
# The data has low collinearity, only Grocery and Detergent Paper sales are highly correlated
# Visually, there are some outliers, especially on the high end, likely some extreme spenders
# Indeed, there are 26 outliers based on Zscore value > 3 = extreme right of the distribution
# Top 20% of customer generate between 50 to 70% of category sales

# Challenge 2 - Data Cleaning and Transformation

If your conclusion from the previous challenge is the data need cleaning/transformation, do it in the cells below. However, if your conclusion is the data need not be cleaned or transformed, feel free to skip this challenge. But if you do choose the latter, please provide rationale.

In [ ]:
# Your code here

customers = pd.read_csv('../data/Wholesale_customer_ data.csv')
customers = customers[(np.abs(stats.zscore(customers)) < 3).all(axis=1)]
customers.info()

In [ ]:
# Your comment here

# I removed the outliers because these are few selected high spenders which should be treated individually and not pollute the cluster analysis
# I have not removed any column because despite the collinearity, keeping both grocery and detergent paper sales is more face valid


# Challenge 3 - Data Preprocessing

One problem with the dataset is the value ranges are remarkably different across various categories (e.g. `Fresh` and `Grocery` compared to `Detergents_Paper` and `Delicassen`). If you made this observation in the first challenge, you've done a great job! This means you not only completed the bonus questions in the previous Supervised Learning lab but also researched deep into [*feature scaling*](https://en.wikipedia.org/wiki/Feature_scaling). Keep on the good work!

Diverse value ranges in different features could cause issues in our clustering. The way to reduce the problem is through feature scaling. We'll use this technique again with this dataset.

#### We will use the `StandardScaler` from `sklearn.preprocessing` and scale our data. Read more about `StandardScaler` [here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).

*After scaling your data, assign the transformed data to a new variable `customers_scale`.*

In [ ]:
# Your import here:

from sklearn import preprocessing

# Your code here:

colnames = [col for col in customers.columns]
scaler = preprocessing.StandardScaler()
customer_scale = scaler.fit_transform(customers)
customer_scale = pd.DataFrame(customer_scale)
customer_scale.columns = colnames
customer_scale

# Challenge 3 - Data Clustering with K-Means

Now let's cluster the data with K-Means first. Initiate the K-Means model, then fit your scaled data. In the data returned from the `.fit` method, there is an attribute called `labels_` which is the cluster number assigned to each data record. What you can do is to assign these labels back to `customers` in a new column called `customers['labels']`. Then you'll see the cluster results of the original data.

In [ ]:
# Your code here:
from sklearn.cluster import KMeans
cluster_model = KMeans()
cluster_model.fit(customer_scale)

In [ ]:
customers['labels'] = cluster_model.predict(customer_scale)
customers.head()

Count the values in `labels`.

In [ ]:
# Your code here:

customers.groupby(['labels']).count()

# Challenge 4 - Data Clustering with DBSCAN

Now let's cluster the data using DBSCAN. Use `DBSCAN(eps=0.5)` to initiate the model, then fit your scaled data. In the data returned from the `.fit` method, assign the `labels_` back to `customers['labels_DBSCAN']`. Now your original data have two labels, one from K-Means and the other from DBSCAN.

In [ ]:
# Your code here

from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=0.5).fit(customer_scale)
clustering
customers['labels_DBSCAN'] = clustering.fit_predict(customer_scale)
customers.head()

Count the values in `labels_DBSCAN`.

In [ ]:
# Your code here

customers.groupby(['labels_DBSCAN']).count()

# Challenge 5 - Compare K-Means with DBSCAN

Now we want to visually compare how K-Means and DBSCAN have clustered our data. We will create scatter plots for several columns. For each of the following column pairs, plot a scatter plot using `labels` and another using `labels_DBSCAN`. Put them side by side to compare. Which clustering algorithm makes better sense?

Columns to visualize:

* `Detergents_Paper` as X and `Milk` as y
* `Grocery` as X and `Fresh` as y
* `Frozen` as X and `Delicassen` as y

Visualize `Detergents_Paper` as X and `Milk` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
def scatter_make(df, x, y):
    fig, [ax1, ax2] = plt.subplots(1,2, figsize = (15,4))
    ax1.scatter(df[x], df['labels'], color='blue')
    ax1.set_title('K-Means')   
    ax1.set_xlabel(x)     
    ax1.set_ylabel('K-Means Clusters')   

    ax2.scatter(df['labels_DBSCAN'], customers[y], color='red')
    ax2.set_title('DBSCAN')
    ax2.set_xlabel('DBSCAN Clusters')
    ax2.set_ylabel(y)

    return plt.show()

In [ ]:
scatter_make(customers, 'Detergents_Paper','Milk')

In [ ]:
scatter_make(customers, 'Grocery','Fresh')

In [ ]:
scatter_make(customers, 'Frozen','Delicassen')

Visualize `Grocery` as X and `Fresh` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
# Your code here:


Visualize `Frozen` as X and `Delicassen` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
# Your code here:

Let's use a groupby to see how the mean differs between the groups. Group `customers` by `labels` and `labels_DBSCAN` respectively and compute the means for all columns.

In [ ]:
# Your code here:

kmean = customers.groupby(['labels']).mean()
DBSCAN = customers.groupby(['labels_DBSCAN']).mean()


In [ ]:
kmean.round()

In [ ]:
# To better assess the clusters, I retrieve the percentile of each mean from the original distribution

percentile_fresh = [stats.percentileofscore(customers['Fresh'],mean) for mean in kmean['Fresh']]
percentile_milk = [stats.percentileofscore(customers['Milk'],mean) for mean in kmean['Milk']]
percentile_grocery = [stats.percentileofscore(customers['Grocery'],mean) for mean in kmean['Grocery']]
percentile_frozen = [stats.percentileofscore(customers['Frozen'],mean) for mean in kmean['Frozen']]
percentile_Detergent = [stats.percentileofscore(customers['Detergents_Paper'],mean) for mean in kmean['Detergents_Paper']]
percentile_delicassen = [stats.percentileofscore(customers['Delicassen'],mean) for mean in kmean['Delicassen']]
channel = kmean['Channel']
region = kmean['Region']


In [ ]:
# I then create a new DF with the above values (+ original Channel and Region for which it makes no sense to calculate the percentile)

percentile_KMean = pd.DataFrame(data=percentile_fresh)
percentile_KMean.columns = ['Fresh']
percentile_KMean['Milk'] = percentile_milk
percentile_KMean['Grocery'] = percentile_grocery
percentile_KMean['Frozen'] = percentile_frozen
percentile_KMean['Detergents_Paper'] = percentile_Detergent
percentile_KMean['Delicassen'] = percentile_delicassen
percentile_KMean['Channel'] = channel
percentile_KMean['Region'] = region
                    

In [ ]:
def color_df(value):
  """
  Colors elements in a dateframe
  green if positive and red if
  negative. Does not color NaN
  values.
  """

  if value < 40:
    color = 'red'
  elif value > 60:
    color = 'green'
  else:
    color = 'black'

  return 'color: %s' % color

In [ ]:
percentile_KMean = percentile_KMean.round()
percentile_KMean.style.applymap(color_df, subset=['Fresh','Milk','Grocery','Frozen','Detergents_Paper','Delicassen'])

In [ ]:
# From the above table, w/o going into the details, you can tell that the clusters have a good logic in differentiating consumers.
# E.g. cl. 0 is very high in Deli sales like # 7 but belongs to different region and channel
# E.g. 5 and 7 are similar but one has very low Fresh sales, the other higher Frozen sales.

In [ ]:
DBSCAN.round()

Which algorithm appears to perform better?

In [ ]:
# DBSCAN identifies only two clusters, differentiating very low spenders across all categories from everyone else.

# Bonus Challenge 2 - Changing K-Means Number of Clusters

As we mentioned earlier, we don't need to worry about the number of clusters with DBSCAN because it automatically decides that based on the parameters we send to it. But with K-Means, we have to supply the `n_clusters` param (if you don't supply `n_clusters`, the algorithm will use `8` by default). You need to know that the optimal number of clusters differs case by case based on the dataset. K-Means can perform badly if the wrong number of clusters is used.

In advanced machine learning, data scientists try different numbers of clusters and evaluate the results with statistical measures (read [here](https://en.wikipedia.org/wiki/Cluster_analysis#External_evaluation)). We are not using statistical measures today but we'll use our eyes instead. In the cells below, experiment with different number of clusters and visualize with scatter plots. What number of clusters seems to work best for K-Means?

In [ ]:
# Your code here

In [ ]:
# Your comment here

# Bonus Challenge 3 - Changing DBSCAN `eps` and `min_samples`

Experiment changing the `eps` and `min_samples` params for DBSCAN. See how the results differ with scatter plot visualization.

In [ ]:
# Your code here

In [ ]:
# Your comment here